<a href="https://colab.research.google.com/github/Deadfish-hk/AcuCount_imageset/blob/main/MMrotate_training_github_v0_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This ipynb script is adapted from the MMrotate colab tutorial in (https://github.com/open-mmlab/mmrotate/blob/main/demo/MMRotate_Tutorial.ipynb).
# Some modifications were made for the script to run seamlessly with the needle training set and configurations.
# After the training, you shall obtain the .pth files under the directory which allows you to undergo further inference and deployment.

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
# Clone the AcuCount Repo for dataset and configs
!git clone https://github.com/Deadfish-hk/AcuCount_imageset.git

In [ ]:
# Download the AcuCount model stored in Google drive for demostration
!pip install gdown
!gdown https://drive.google.com/uc?id=11qKHCZYwAelya_bHDej4xDTKu7k9eb8z

In [ ]:
# Download the AcuCount training imageset stored in Google drive for model training
!gdown https://drive.google.com/uc?id=1_7vEBMvhG_Eu6CcPIFVV3PnjLkLGAFtr

In [ ]:
# Install dependencies: (use cu116 because colab has CUDA 11.6)
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 -f https://download.pytorch.org/whl/torch_stable.html

# Install mmcv-full and mmdetection thus we could use CUDA operators
!pip3 install -U openmim
!mim install mmdet\<3.0.0
!mim install mmcv-full==1.6.2

# Install mmrotate
!git clone --branch v0.3.4 https://github.com/open-mmlab/mmrotate.git
%cd mmrotate
!pip install -e .

In [ ]:
from mmcv import collect_env
collect_env()

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
# Download the pre-trained Oriented R-CNN weights
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmrotate/v0.1.0/oriented_rcnn/oriented_rcnn_r50_fpn_1x_dota_le90/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth \
      -O checkpoints/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth

In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmrotate.models import build_detector

# Choose to use a config and initialize the detector
config = '/content/AcuCount_imageset/AcuCount_detection_model_config.py'
# Setup a checkpoint file to load
checkpoint = '/content/AcuCount_detection_model.pth'

# Set the device to be used for evaluation
device='cuda:0'

# Load the config
config = mmcv.Config.fromfile(config)
# Set pretrained to be None since we do not need pretrained model here
config.model.pretrained = None

# Initialize the detector
model = build_detector(config.model)

# Load checkpoint
checkpoint = load_checkpoint(model, checkpoint, map_location=device)

# Set the classes of models for inference
model.CLASSES = checkpoint['meta']['CLASSES']

# We need to set the model's cfg for inference
model.cfg = config

# Convert the model to GPU
model.to(device)
# Convert the model into evaluation mode
model.eval()

In [ ]:
# Lets try visualize an example image

img = mmcv.imread('/content/AcuCount_imageset/Validation_imageset/Copper_and_Silver_validation/Copper_and_Silver_0030.jpg')
result = inference_detector(model, img)
show_result_pyplot(model, img, result, score_thr=0.5, palette='dota')

In [ ]:
!unzip /content/AcuCount_training_dota.zip

In [ ]:
# Check the directory structure of the data by installing tree
# Note that the imageset contains 590 training images and 79 testing images, So a total of 1338 files including annotation txt files.

!apt-get -q install tree
!tree /content/mmrotate/final_needle_combined_dota

In [ ]:
# Check the label of a single image
!cat final_needle_combined_dota/train/training_0429.txt

In [ ]:
from mmrotate.datasets.builder import ROTATED_DATASETS
from mmrotate.datasets.dota import DOTADataset


@ROTATED_DATASETS.register_module()
class TinyDataset(DOTADataset):
    CLASSES = ('needle', )     # Input the name of your class label.

In [ ]:
# Import config file from AcuCount Repo

from mmcv import Config
cfg = Config.fromfile('/content/AcuCount_imageset/AcuCount_detection_model_config.py')

In [ ]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'TinyDataset'
cfg.data_root = 'final_needle_combined_dota/'

cfg.data.test.type = 'TinyDataset'
cfg.data.test.data_root = 'final_needle_combined_dota/'
cfg.data.test.ann_file = 'val'
cfg.data.test.img_prefix = 'images'

cfg.data.train.type = 'TinyDataset'
cfg.data.train.data_root = 'final_needle_combined_dota/'
cfg.data.train.ann_file = 'train'
cfg.data.train.img_prefix = 'images'

cfg.data.val.type = 'TinyDataset'
cfg.data.val.data_root = 'final_needle_combined_dota/'
cfg.data.val.ann_file = 'val'
cfg.data.val.img_prefix = 'images'

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = 'checkpoints/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

cfg.optimizer.lr = 0.01
cfg.lr_config.step = [53, 72]
cfg.lr_config.warmup = None
cfg.runner.max_epochs = 80
cfg.log_config.interval = 10

cfg.model.rpn_head.anchor_generator.ratios = [0.0625, 0.125, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 12.0, 16.0] #0.5, 1.0, 2.0 # [0.0625, 0.125, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0]

cfg.model.test_cfg.rcnn.nms.iou_thr = 0.3 # 0.25 # 0.35 #optimal 0.3

cfg.test_pipeline[1].img_scale = (960, 1280)

cfg.data.val.pipeline[1].img_scale = (960, 1280)

cfg.data.test.pipeline[1].img_scale = (960, 1280)

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 10
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(0, 1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

# We can initialize the logger for training and have a look
# at the final config used for training
#print(f'Config:\n{cfg.pretty_text}')

In [ ]:
import os.path as osp
import mmcv

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
cfg.device = 'cuda'
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
cfg.device = 'cuda'
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
# Validate the dataset with display of model inference time

import glob
from mmdet.apis import inference_detector, show_result_pyplot
from mmrotate.models import build_detector
import time

import logging

# Logging module

logger = logging.getLogger()
fhandler = logging.FileHandler(filename='/content/timeevent.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

import numpy as np

# Counting function

def counting(result):
  count = 0
  count_order = 0
  count_list = result[0]
  for item in np.nditer(count_list):
    box_score = count_list[count_order][5]
    if box_score > 0.5:
      count = count + 1
      count_order = count_order + 1
      if count_order == len(count_list):
        break
    if box_score < 0.5:
      count_order = count_order + 1
      if count_order == len(count_list):
        break
  return count

In [ ]:
# Validate the dataset with model inference time
# Change the directory path for detection of different needles (Copper, silver, copper_and_silver)

png_order = 0
png_list = glob.glob('/content/AcuCount_imageset/Validation_imageset/Copper_and_Silver_validation/*jpg')
for item in png_list:
  png_filename = png_list[png_order]
  saved_filename = png_filename[-8:]
  img = mmcv.imread(f'{png_filename}')
  start_time = time.time()
  result = inference_detector(model, img)
  elapsed_time = time.time() - start_time
  logger.info(f"Inference done. elapsed time:{elapsed_time}")
  counting_result = counting(result)
  show_result_pyplot(model, img, result, score_thr=0.5, out_file=f'/content/Final_testing/copper_and_silver/{counting_result}_{saved_filename}')
  png_order = png_order + 1
